In [1]:
import auxiliary as aux

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from lightgbm import LGBMRegressor

## best approach

In [21]:
# Kaggle: 0.91069
predicted = aux.ImputeHelper(
    (aux.ml_impute, aux.subcol[4], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29), 
                                    'max_fill_nan_count': np.inf, 'max_train_nan_count': np.inf}),
    (aux.simplestat, aux.subcol[1] + aux.subcol[3], SimpleImputer(strategy='mean')),
).run(aux.data, inherit=True)

100%|██████████| 15/15 [23:55<00:00, 95.71s/it] 



ML Imputer avg. score: 0.5946655547740444


100%|██████████| 40/40 [00:00<00:00, 41.11it/s]


SimpleStat avg. score: 0.9655820744615067


## tests

In [12]:
# # fill all NaN, train on notNA only (0.98242)
# predicted = aux.ImputeHelper(
#     (aux.ml_impute, aux.subcol[4], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29), 
#                                     'max_fill_nan_count': np.inf, 'max_train_nan_count': 0}),
#     (aux.simplestat, 'all', SimpleImputer(strategy='mean')),
# ).run(aux.data, inherit=True)

In [5]:
# # fill rows with only 1 NaN, train on notNA only; other NaN fill with mean (1.02633)
# predicted = aux.ImputeHelper(
#     (aux.ml_impute, aux.subcol[4], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29), 
#                                     'max_fill_nan_count': 1, 'max_train_nan_count': 0}),
#     (aux.simplestat, 'all', SimpleImputer(strategy='mean')),
# ).run(aux.data, inherit=True)

In [11]:
# # fill rows with only 1 NaN, train on whole data; other NaN fill with mean (1.03272)
# predicted = aux.ImputeHelper(
#     (aux.ml_impute, aux.subcol[4], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29), 
#                                     'max_fill_nan_count': 1, 'max_train_nan_count': np.inf}),
#     (aux.simplestat, 'all', SimpleImputer(strategy='mean')),
# ).run(aux.data, inherit=True)

In [ ]:
# # fill rows with only 1 NaN, train on notNA only; other NaN fill with group mean
# predicted = aux.ImputeHelper(
#     (aux.ml_impute, aux.subcol[4], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29), 
#                                     'max_fill_nan_count': 1, 'max_train_nan_count': 0}),
#     (aux.groupstat, aux.subcol[2] + aux.subcol[4], {'gcol': ['F_2_2']}),       # F_2_2 nunique = 11, F_2_24 & F_2_10 nquniue = 18
# ).run(aux.data, inherit=True)

In [2]:
# fill all NaN with cosine similars
predicted = aux.ImputeHelper(
    (aux.cosine_stats, 'all', {'k': 5000}),
).run(aux.data, inherit=True)

Chunks: 6353


  0%|          | 24/6353 [00:34<2:58:22,  1.69s/it] 

## subsets 1

In [6]:
# fill all NaN, train on whole data
predicted = aux.ImputeHelper(
    (aux.ml_impute, aux.subcol[1], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=100, num_leaves=23), 
                                    'max_fill_nan_count': np.inf, 'max_train_nan_count': np.inf}),
).run(aux.data, inherit=True)

100%|██████████| 15/15 [00:54<00:00,  3.61s/it]



ML Imputer avg. score: 0.9438455290350334


In [7]:
# fill all NaN, train on notNA only
predicted = aux.ImputeHelper(
    (aux.ml_impute, aux.subcol[1], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=100, num_leaves=23), 
                                    'max_fill_nan_count': np.inf, 'max_train_nan_count': 0}),
).run(aux.data, inherit=True)

100%|██████████| 15/15 [00:31<00:00,  2.08s/it]



ML Imputer avg. score: 0.9433812251767555


In [8]:
# fill rows with only 1 NaN, train on whole data; other fill with mean
predicted = aux.ImputeHelper(
    (aux.ml_impute, aux.subcol[1], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=100, num_leaves=23), 
                                    'max_fill_nan_count': 1, 'max_train_nan_count': np.inf}),
    (aux.simplestat, aux.subcol[1], SimpleImputer(strategy='mean')),
).run(aux.data, inherit=True)

100%|██████████| 15/15 [00:49<00:00,  3.31s/it]



ML Imputer avg. score: 0.9438455290350333


100%|██████████| 15/15 [00:00<00:00, 50.85it/s]


SimpleStat avg. score: 0.9390547413431811


In [9]:
# fill rows with only 1 NaN, train on notNA only
predicted = aux.ImputeHelper(
    (aux.ml_impute, aux.subcol[1], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=100, num_leaves=23), 
                                    'max_fill_nan_count': 1, 'max_train_nan_count': 0}),
    (aux.simplestat, aux.subcol[1], SimpleImputer(strategy='mean')),
).run(aux.data, inherit=True)

100%|██████████| 15/15 [00:26<00:00,  1.73s/it]



ML Imputer avg. score: 0.9433812251767555


100%|██████████| 15/15 [00:00<00:00, 51.47it/s]


SimpleStat avg. score: 0.9390555595149658


## submission

In [4]:
aux.save_submission(predicted)

Still contain NaN: False


,row-col,value
0,0-F_1_14,-0.000905
1,0-F_3_23,0.000365
2,1-F_3_24,-0.000817
3,2-F_1_2,0.000551
4,2-F_4_2,0.324838


In [ ]:
#